# Get rows with only latest transaction

- Use the zillow dataset
- Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.

- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.

- Only include properties that include a latitude and longitude value.


### Thoughts
- Isolate a dataframe of the rows with no duplicated parcelids
- Isolate a dataframe w/ duplicated parcelids
    - Then filter to have only the most recent transaction data
- Concatenate those two dataframes together
- Profit!

In [1]:
import pandas as pd
from env import get_db_url
url = get_db_url('zillow')

In [2]:
sql = """
select *
from properties_2017
join predictions_2017 using(parcelid)
where latitude is not null and longitude is not null
and transactiondate <= '2017-12-31'
"""

df = pd.read_sql(sql, url)
df

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,14297519,1727539,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13,0,0.025595,2017-01-01
1,17052889,1387261,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13,1,0.055619,2017-01-01
2,14186244,11677,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13,2,0.005383,2017-01-01
3,12177905,2288172,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,3,-0.103410,2017-01-01
4,10887214,1970746,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,119407.0,2016.0,45726.0,1533.89,None,NaN,6.037124e+13,4,0.006940,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,10833991,2864704,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,379000.0,2016.0,114000.0,4685.34,None,NaN,6.037132e+13,77608,-0.002245,2017-09-20
77575,11000655,673515,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,354621.0,2016.0,283704.0,4478.43,None,NaN,6.037101e+13,77609,0.020615,2017-09-20
77576,17239384,2968375,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,67205.0,2016.0,16522.0,1107.48,None,NaN,6.111008e+13,77610,0.013209,2017-09-21
77577,12773139,1843709,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,49546.0,2016.0,16749.0,876.43,None,NaN,6.037434e+13,77611,0.037129,2017-09-21


In [3]:
# Rows with no duplicates
no_duplicates = df[~df.parcelid.duplicated()]
no_duplicates

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,14297519,1727539,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13,0,0.025595,2017-01-01
1,17052889,1387261,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13,1,0.055619,2017-01-01
2,14186244,11677,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13,2,0.005383,2017-01-01
3,12177905,2288172,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,3,-0.103410,2017-01-01
4,10887214,1970746,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,119407.0,2016.0,45726.0,1533.89,None,NaN,6.037124e+13,4,0.006940,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,10833991,2864704,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,379000.0,2016.0,114000.0,4685.34,None,NaN,6.037132e+13,77608,-0.002245,2017-09-20
77575,11000655,673515,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,354621.0,2016.0,283704.0,4478.43,None,NaN,6.037101e+13,77609,0.020615,2017-09-20
77576,17239384,2968375,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,67205.0,2016.0,16522.0,1107.48,None,NaN,6.111008e+13,77610,0.013209,2017-09-21
77577,12773139,1843709,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,49546.0,2016.0,16749.0,876.43,None,NaN,6.037434e+13,77611,0.037129,2017-09-21


In [4]:
# 198 duplicated parcelids
df.parcelid.duplicated().sum()

198

In [5]:
df.groupby('parcelid').parcelid.count()[df.groupby('parcelid').parcelid.count() > 1]

parcelid
10722858     2
10732347     2
10739478     2
10744507     2
10753427     2
            ..
17251843     2
17280166     2
17282392     2
17295416     2
162960529    2
Name: parcelid, Length: 195, dtype: int64

In [6]:
# Number of times each parcelid shows in this dataframe
df.groupby('parcelid').parcelid.count().sort_values(ascending=False)

parcelid
10857130     3
11991059     3
12612211     3
11739891     2
11577176     2
            ..
11876038     1
11876013     1
11875917     1
11875901     1
167689317    1
Name: parcelid, Length: 77381, dtype: int64

In [7]:
# Generate a series of unique parcelids that have been duplicated
duplicate_parcelids = df[df.parcelid.duplicated()].parcelid
duplicate_parcelids

117      11393337
624      14634203
1017     11721753
1246     11289917
1732     11637029
           ...   
59986    13066981
62214    10852812
63107    12136147
64253    17282392
64918    10984080
Name: parcelid, Length: 198, dtype: int64

In [8]:
# All the rows with a parcelid showing up more than once
rows_with_duplicate_parcelids = df[df.parcelid.isin(duplicate_parcelids)]
rows_with_duplicate_parcelids

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
116,11393337,2463969,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,43439.0,2016.0,22755.0,756.94,Y,14.0,6.037235e+13,116,0.015172,2017-01-03
117,11393337,2463969,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,43439.0,2016.0,22755.0,756.94,Y,14.0,6.037235e+13,117,0.086137,2017-06-08
623,14634203,2026522,1.0,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,289445.0,2016.0,213516.0,3490.16,None,NaN,6.059001e+13,624,-0.019491,2017-01-04
624,14634203,2026522,1.0,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,289445.0,2016.0,213516.0,3490.16,None,NaN,6.059001e+13,625,-0.061973,2017-08-04
1016,11721753,616260,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,205123.0,2016.0,163175.0,2627.48,None,NaN,6.037220e+13,1017,-0.011052,2017-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63107,12136147,2407178,NaN,NaN,NaN,2.0,3.0,NaN,5.0,2.0,...,775383.0,2016.0,608190.0,9103.61,None,NaN,6.037431e+13,63132,-0.078758,2017-08-31
64252,17282392,2938730,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,498684.0,2016.0,249342.0,5258.54,None,NaN,6.111007e+13,64277,-0.002968,2017-08-07
64253,17282392,2938730,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,498684.0,2016.0,249342.0,5258.54,None,NaN,6.111007e+13,64278,0.901074,2017-08-25
64917,10984080,2876815,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,354000.0,2016.0,141000.0,4347.32,Y,15.0,6.037104e+13,64943,0.001824,2017-08-09


In [9]:
# Obtain the latest transaction date for each parcelid
latest = pd.DataFrame(rows_with_duplicate_parcelids.groupby("parcelid").transactiondate.max())
latest["parcelid"] = latest.index
latest = latest.reset_index(drop=True)
latest

,transactiondate,parcelid
0,2017-07-28,10722858
1,2017-07-25,10732347
2,2017-03-31,10739478
3,2017-08-31,10744507
4,2017-03-17,10753427
...,...,...
190,2017-06-22,17251843
191,2017-06-15,17280166
192,2017-08-25,17282392
193,2017-05-16,17295416


In [10]:
only_latest = rows_with_duplicate_parcelids.merge(latest, on=["transactiondate", "parcelid"])
only_latest

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,11393337,2463969,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,43439.0,2016.0,22755.0,756.94,Y,14.0,6.037235e+13,117,0.086137,2017-06-08
1,14634203,2026522,1.0,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,289445.0,2016.0,213516.0,3490.16,None,NaN,6.059001e+13,625,-0.061973,2017-08-04
2,11721753,616260,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,205123.0,2016.0,163175.0,2627.48,None,NaN,6.037220e+13,1018,0.017785,2017-07-21
3,11289917,2061546,1.0,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,-0.362001,2017-06-23
4,11637029,2554497,1.0,NaN,NaN,2.0,3.0,NaN,9.0,2.0,...,810694.0,2016.0,554156.0,9407.01,Y,13.0,6.037701e+13,1734,-0.093399,2017-07-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,13066981,2008746,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,366160.0,2016.0,174133.0,4872.87,None,NaN,6.037403e+13,60010,-0.043423,2017-09-01
191,10852812,492024,NaN,NaN,NaN,7.0,11.0,NaN,8.0,7.0,...,5373376.0,2016.0,1923376.0,64396.38,None,NaN,NaN,62239,2.237069,2017-08-31
192,12136147,2407178,NaN,NaN,NaN,2.0,3.0,NaN,5.0,2.0,...,775383.0,2016.0,608190.0,9103.61,None,NaN,6.037431e+13,63132,-0.078758,2017-08-31
193,17282392,2938730,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,498684.0,2016.0,249342.0,5258.54,None,NaN,6.111007e+13,64278,0.901074,2017-08-25


In [11]:
final = pd.concat([only_latest, no_duplicates])
final

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,11393337,2463969,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,43439.0,2016.0,22755.0,756.94,Y,14.0,6.037235e+13,117,0.086137,2017-06-08
1,14634203,2026522,1.0,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,289445.0,2016.0,213516.0,3490.16,None,NaN,6.059001e+13,625,-0.061973,2017-08-04
2,11721753,616260,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,205123.0,2016.0,163175.0,2627.48,None,NaN,6.037220e+13,1018,0.017785,2017-07-21
3,11289917,2061546,1.0,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,-0.362001,2017-06-23
4,11637029,2554497,1.0,NaN,NaN,2.0,3.0,NaN,9.0,2.0,...,810694.0,2016.0,554156.0,9407.01,Y,13.0,6.037701e+13,1734,-0.093399,2017-07-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,10833991,2864704,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,379000.0,2016.0,114000.0,4685.34,None,NaN,6.037132e+13,77608,-0.002245,2017-09-20
77575,11000655,673515,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,354621.0,2016.0,283704.0,4478.43,None,NaN,6.037101e+13,77609,0.020615,2017-09-20
77576,17239384,2968375,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,67205.0,2016.0,16522.0,1107.48,None,NaN,6.111008e+13,77610,0.013209,2017-09-21
77577,12773139,1843709,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,49546.0,2016.0,16749.0,876.43,None,NaN,6.037434e+13,77611,0.037129,2017-09-21


In [12]:
final.parcelid.duplicated().sum()

195

In [13]:
no_duplicates.parcelid.duplicated().sum()

0

In [14]:
only_latest.parcelid.duplicated().sum()

0

In [15]:
# Looks like I still have an issue with the logic of this approach.
# TODO: revisit my approach
final[final.parcelid.duplicated()]

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
116,11393337,2463969,NaN,NaN,NaN,3.0,3.0,NaN,4.0,3.0,...,43439.0,2016.0,22755.0,756.94,Y,14.0,6.037235e+13,116,0.015172,2017-01-03
623,14634203,2026522,1.0,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,289445.0,2016.0,213516.0,3490.16,None,NaN,6.059001e+13,624,-0.019491,2017-01-04
1016,11721753,616260,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,205123.0,2016.0,163175.0,2627.48,None,NaN,6.037220e+13,1017,-0.011052,2017-01-05
1245,11289917,2061546,1.0,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1247,0.227903,2017-01-06
1731,11637029,2554497,1.0,NaN,NaN,2.0,3.0,NaN,9.0,2.0,...,810694.0,2016.0,554156.0,9407.01,Y,13.0,6.037701e+13,1733,-0.024199,2017-01-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59985,13066981,2008746,1.0,NaN,NaN,2.0,4.0,NaN,8.0,2.0,...,366160.0,2016.0,174133.0,4872.87,None,NaN,6.037403e+13,60009,0.016734,2017-07-24
62213,10852812,492024,NaN,NaN,NaN,7.0,11.0,NaN,8.0,7.0,...,5373376.0,2016.0,1923376.0,64396.38,None,NaN,NaN,62238,2.027777,2017-07-28
63106,12136147,2407178,NaN,NaN,NaN,2.0,3.0,NaN,5.0,2.0,...,775383.0,2016.0,608190.0,9103.61,None,NaN,6.037431e+13,63131,0.056067,2017-08-01
64252,17282392,2938730,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,498684.0,2016.0,249342.0,5258.54,None,NaN,6.111007e+13,64277,-0.002968,2017-08-07
